In [1]:
# Let's explore some agents

In [2]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm.invoke("How many letters in the word educa?")

AIMessage(content='There are 6 letters in the word "educa".')

In [3]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Return the length of a word."""
    return len(word)

tools = [get_word_length]

In [4]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're a powerful assistant, but bad at calculating lengths of words."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [17]:
from langchain.tools.render import format_tool_to_openai_function

# Binding passes in the tools each time we invoke the model
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

# Thoughts: Does this use up the context?

In [22]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [29]:
response = agent.invoke({"input": "How many letters in the word educa?", "intermediate_steps": []})
response

AgentActionMessageLog(tool='get_word_length', tool_input={'word': 'educa'}, log="\nInvoking: `get_word_length` with `{'word': 'educa'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "word": "educa"\n}', 'name': 'get_word_length'}})])

In [41]:
# But we need a runtime in order to be able to handle the agents and executing functions.

from langchain.schema.agent import AgentFinish

# The world's most basic runtime
user_input = "how many letters in the word educa? and how many letters in the word praya?"
intermediate_steps = []
while True:
    output = agent.invoke(
        {
            "input": user_input,
            "intermediate_steps": intermediate_steps,
        }
    )

    # If we encounter the finish
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:  # If we don't, we continue
        print(f"OUTPUT: {output}")
        print(f"TOOL NAME: {output.tool}")
        print(f"TOOL INPUT: {output.tool_input}")
        tool = {"get_word_length": get_word_length}[output.tool]  # this is ugly, but it'll do for now because we only have one tool
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))
print(final_result)

OUTPUT: tool='get_word_length' tool_input={'word': 'educa'} log="\nInvoking: `get_word_length` with `{'word': 'educa'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "word": "educa"\n}', 'name': 'get_word_length'}})]
TOOL NAME: get_word_length
TOOL INPUT: {'word': 'educa'}
OUTPUT: tool='get_word_length' tool_input={'word': 'praya'} log="\nInvoking: `get_word_length` with `{'word': 'praya'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "word": "praya"\n}', 'name': 'get_word_length'}})]
TOOL NAME: get_word_length
TOOL INPUT: {'word': 'praya'}
The word "educa" has 5 letters, and the word "praya" also has 5 letters.


In [42]:
# Luckily there's ome included executors
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [44]:
agent_executor.invoke(
    {"input": "how many letters in the world educa? What about the word superaya?"}
)



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5
Invoking: `get_word_length` with `{'word': 'superaya'}`


8The word "educa" has 5 letters, while the word "superaya" has 8 letters.

> Finished chain.


{'input': 'how many letters in the world educa? What about the word superaya?',
 'output': 'The word "educa" has 5 letters, while the word "superaya" has 8 letters.'}

In [45]:
# All right, let's add memory to make things stateful.

from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are a powerful assistant, but bad at calculating lengths of words."
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

In [46]:
# Let's keep track of the chat history
from langchain.schema.messages import AIMessage, HumanMessage

chat_history = []

In [47]:
# Let's put it together
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [49]:
# Let's do some magic
input1 = "how many letters in the word educa?"
result = agent_executor.invoke(
    {
        "input": input1, "chat_history": chat_history
    }
)

chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)

# Execute a second step
agent_executor.invoke(
    {
        "input": "is that a real word?",
        "chat_history": chat_history
    }
)



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5The word "educa" is not a real word. It was just used as an example to calculate the number of letters in a word.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'The word "educa" is not a real word. It was just used as an example to calculate the number of letters in a word.'}